In [1]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import SequentialFeatureSelector as SFS

In [18]:
data = pd.read_csv('../precleaned-datasets/dataset_2.csv')
data

,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,...,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109
0,4.532710,3.280834,1.798248e+01,4.404259,2.349910,0.603264,2.784655,0.323146,12.009691,0.139346,...,2.079066,6.748819,2.941445,18.360496,17.726613,7.774031,1.473441,1.973832e+00,0.976806,2.541417
1,5.821374,12.098722,1.330915e+01,4.125599,1.045386,1.832035,1.833494,0.709090,8.652883,0.102757,...,2.479789,7.795290,3.557890,17.383378,15.193423,8.263673,1.878108,5.679390e-01,1.018818,1.416433
2,1.938776,7.952752,9.726712e-01,3.459267,1.935782,0.621463,2.338139,0.344948,9.937850,11.691283,...,1.861487,6.130886,3.401064,15.850471,14.620599,6.849776,1.098210,1.959183e+00,1.575493,1.857893
3,6.020690,9.900544,1.786964e+01,4.366715,1.973693,2.026012,2.853025,0.674847,11.816859,0.011151,...,1.340944,7.240058,2.417235,15.194609,13.553772,7.229971,0.835158,2.234482e+00,0.946170,2.700606
4,3.909506,10.576516,9.341910e-01,3.419572,1.871438,3.340811,1.868282,0.439865,13.585620,1.153366,...,2.738095,6.565509,4.341414,15.893832,11.929787,6.954033,1.853364,5.110269e-01,2.599562,0.811364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,5.392593,1.336383,8.212083e+00,2.966147,2.387417,3.019191,2.289753,0.696969,9.218114,0.451762,...,2.393518,5.486381,2.420444,18.140092,16.388475,5.538263,0.982295,1.851852e+00,3.642670,1.264955
49996,3.544616,5.376492,-5.253303e-07,3.469798,3.473573,4.555753,1.238703,0.332744,15.448531,4.587770,...,2.452819,8.320470,4.880449,17.023601,14.157572,8.354363,1.852349,3.015394e-01,1.312911,1.473906
49997,4.266667,7.618359,1.728767e+01,3.241785,1.264538,2.392204,2.181862,0.703589,13.643899,1.555709,...,2.163462,6.931792,4.603708,18.112407,17.251750,7.588980,2.594008,2.230302e+00,2.525820,2.360954
49998,5.130579,6.749948,1.060437e+01,3.578128,2.770539,2.089231,2.698422,1.165991,11.427852,0.020571,...,0.938087,5.772450,2.003628,17.247582,16.112546,4.295269,0.847949,-5.341621e-07,0.252078,1.767468


In [19]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['target']),
                                                     data['target'],
                                                       test_size=0.2,
                                                         random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(40000, 108) (10000, 108) (40000,) (10000,)


In [20]:
len(X_train.columns)  

108

In [21]:
# remove correlated features to reduce feature space
correlated_features = set()
correlation_matrix = data.corr()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

print(f"Correlated features: {correlated_features}")
X_train = X_train.drop(columns=correlated_features)
X_test = X_test.drop(columns=correlated_features)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Correlated features: {'var_54', 'var_29', 'var_64', 'var_32', 'var_74', 'var_106', 'var_24', 'var_102', 'var_109', 'var_85', 'var_59', 'var_87', 'var_95', 'var_42', 'var_97', 'var_44', 'var_39', 'var_101', 'var_104', 'var_108', 'var_17', 'var_66', 'var_94', 'var_76', 'var_105', 'var_88', 'var_43', 'var_72', 'var_57', 'var_33', 'var_80', 'var_28', 'var_100', 'var_75', 'var_84', 'var_70'}
(40000, 72) (10000, 72) (40000,) (10000,)


In [22]:
len(X_train.columns)  

72

In [26]:
sfs = SFS(RandomForestClassifier(n_estimators=100, random_state=42),
        n_features_to_select=10,  # -->  Number of features to select
          direction='forward',   #  --> Forward selection
          scoring='roc_auc', # --> Use ROC AUC as the scoring metric
          cv=3)           # --> 5-fold cross-validation


In [27]:
sfs.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
selected_feat = sfs.get_feature_names_out()
print(selected_feat)

In [ ]:
X_train_selected = sfs.transform(X_train)
X_test_selected = sfs.transform(X_test)

print(X_train_selected.shape, X_test_selected.shape, y_train.shape, y_test.shape)